In [1]:
from segmentation_models import Unet
from segmentation_models.backbones import get_preprocessing
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras import backend as K
from keras.utils import to_categorical
K.set_image_data_format('channels_last')
from tqdm import tqdm
K.clear_session()
# from google.colab import drive

Using TensorFlow backend.
C:\Users\sm00th\AppData\Local\conda\conda\envs\tensorflow_gpuenv\lib\site-packages\classification_models\resnext\__init__.py:4: UserWarning: Current ResNext models are deprecated, use keras.applications ResNeXt models
  warnings.warn('Current ResNext models are deprecated, '


In [2]:
BACKBONE = 'resnet34'
preprocess_input = get_preprocessing(BACKBONE)

In [3]:
# drive.mount('/content/drive')

# !unzip drive/My\ Drive/Colab\ Notebooks/hw5/dataset1.zip
# ! mv dataset1 drive/My\ Drive/Colab\ Notebooks/hw5

In [4]:
IM_WIDTH, IM_HEIGHT = 360, 480
RESOLUT = (IM_WIDTH, IM_HEIGHT)
BATCH_SIZE = 8

# TR_DIR = r'/drive/My\ Drive/Colab\ Notebooks/hw5/dataset1/annotations_prepped_train'

In [5]:
# !ls drive/My\ Drive/Colab\ Notebooks/hw5/dataset1  

In [6]:
images_prepped_train = r'C:\Users\sm00th\Dropbox\Python\Neural Univercity\hw5\dataset1\images_prepped_train'
# images_prepped_train = r'drive/My Drive/Colab Notebooks/hw5/dataset1/images_prepped_train'

images_prepped_test = r'C:\Users\sm00th\Dropbox\Python\Neural Univercity\hw5\dataset1\images_prepped_test'

annotations_prepped_train = r'C:\Users\sm00th\Dropbox\Python\Neural Univercity\hw5\dataset1\annotations_prepped_train'
# annotations_prepped_train = r'drive/My Drive/Colab Notebooks/hw5/dataset1/annotations_prepped_train'

annotations_prepped_test = r'C:\Users\sm00th\Dropbox\Python\Neural Univercity\hw5\dataset1\annotations_prepped_test'

# annotations_prepped_train_list = os.listdir(annotations_prepped_train)

In [7]:
def read_ims(path):
    ims = []
    files = os.listdir(path)
    for file in files:
        ims.append(plt.imread(os.path.join(path, file)))
    ims = preprocess_input(ims)
    ims = np.array(ims)
    return ims

def read_labels(path):
    files = os.listdir(path)   
    im = np.empty((len(files), 360, 480, 6))
    for n, file in enumerate(files):
        i = plt.imread(os.path.join(path, file))*255
        i = i.astype('int')
        i = np.where(i >=5, 5, i)
        im[n] = to_categorical(i)
    return im

In [8]:
train_ims = read_ims(os.path.join(images_prepped_train, 'all'))
train_labels = read_labels(os.path.join(annotations_prepped_train, 'all'))
test_ims = read_ims(os.path.join(images_prepped_test, 'all'))
test_labels = read_labels(os.path.join(annotations_prepped_test, 'all'))

In [9]:
train_ims.shape, train_labels.shape, test_ims.shape, test_labels.shape

((367, 360, 480, 3),
 (367, 360, 480, 6),
 (101, 360, 480, 3),
 (101, 360, 480, 6))

In [10]:
datagen = ImageDataGenerator()

In [11]:
train_generator = datagen.flow(train_ims, train_labels,
                        batch_size=BATCH_SIZE)

test_generator = datagen.flow(test_ims, test_labels,
                        batch_size=BATCH_SIZE,
                        shuffle=False)

In [12]:
train_generator.next()[0].shape

(8, 360, 480, 3)

In [13]:
test_generator.next()[1].shape

(8, 360, 480, 6)

In [14]:
class UNet():

    def __init__(self):

        print ('build UNet ...')



    def get_crop_shape(self, target, refer):

        # width, the 3rd dimension

        cw = (target.get_shape()[2] - refer.get_shape()[2]).value

        assert (cw >= 0)

        if cw % 2 != 0:

            cw1, cw2 = int(cw/2), int(cw/2) + 1

        else:

            cw1, cw2 = int(cw/2), int(cw/2)

        # height, the 2nd dimension

        ch = (target.get_shape()[1] - refer.get_shape()[1]).value

        assert (ch >= 0)

        if ch % 2 != 0:

            ch1, ch2 = int(ch/2), int(ch/2) + 1

        else:

            ch1, ch2 = int(ch/2), int(ch/2)



        return (ch1, ch2), (cw1, cw2)



    def create_model(self, img_shape, num_class):



        concat_axis = 3

        inputs = layers.Input(shape = img_shape)



        conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_1')(inputs)

        conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)

        pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)

        conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)

        pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)



        conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)

        conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)

        pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)



        conv4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)

        conv4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)

        pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)



        conv5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)

        conv5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)



        up_conv5 = layers.UpSampling2D(size=(2, 2))(conv5)

        ch, cw = self.get_crop_shape(conv4, up_conv5)

        crop_conv4 = layers.Cropping2D(cropping=(ch,cw))(conv4)

        up6 = layers.concatenate([up_conv5, crop_conv4], axis=concat_axis)

        conv6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up6)

        conv6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)



        up_conv6 = layers.UpSampling2D(size=(2, 2))(conv6)

        ch, cw = self.get_crop_shape(conv3, up_conv6)

        crop_conv3 = layers.Cropping2D(cropping=(ch,cw))(conv3)

        up7 = layers.concatenate([up_conv6, crop_conv3], axis=concat_axis) 

        conv7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up7)

        conv7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)



        up_conv7 = layers.UpSampling2D(size=(2, 2))(conv7)

        ch, cw = self.get_crop_shape(conv2, up_conv7)

        crop_conv2 = layers.Cropping2D(cropping=(ch,cw))(conv2)

        up8 = layers.concatenate([up_conv7, crop_conv2], axis=concat_axis)

        conv8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up8)

        conv8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)



        up_conv8 = layers.UpSampling2D(size=(2, 2))(conv8)

        ch, cw = self.get_crop_shape(conv1, up_conv8)

        crop_conv1 = layers.Cropping2D(cropping=(ch,cw))(conv1)

        up9 = layers.concatenate([up_conv8, crop_conv1], axis=concat_axis)

        conv9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up9)

        conv9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)



        ch, cw = self.get_crop_shape(inputs, conv9)

        conv9 = layers.ZeroPadding2D(padding=((ch[0], ch[1]), (cw[0], cw[1])))(conv9)

        conv10 = layers.Conv2D(num_class, (1, 1))(conv9)



        model = models.Model(inputs=inputs, outputs=conv10)



        return model

In [15]:
init_colors = [[np.random.randint(0, 256) for _ in range(3)] for _ in range(6)]

def target_preproc(im):
    im[:, :] = np.where(im[:, :] >= 5, 6, im[:, :])
    
    for cl in range(6):
        for ch in range(im.shape[2]):
            im[:, :, ch] = np.where(im[:, :, ch]==cl, init_colors[cl][ch], im[:, :, ch])
            
    im = np.clip(im, 0, 255) / 255
    return im


def ohe_classes(im):
    im = im[:, :, 0]
    return im

In [16]:
# for k in range(8):
#     plt.imshow(a[k])
#     plt.show()

In [23]:
def init_model():
    # Модель от Павла
#     model = Unet('resnet34', 
#                                      input_shape=(IM_WIDTH, IM_HEIGHT, 3), 
#                                      intencoder_weights='imagenet', 
#                                      classes=6, activation='softmax')
    
    # Модель из нета с кропом
    unet = UNet()
    unet = unet.create_model(img_shape=(IM_WIDTH, IM_HEIGHT, 3), 
                  num_class=6)

    return unet

def train_model(model, epochs=100):
    model.compile('Adam', 
#                  loss='categorical_crossentropy',
                 loss='bce_jaccard_loss',
#                  metrics=['categorical_accuracy']
                 metrics=['iou_score']
                )
    
    history = model.fit_generator(train_generator,
                                  steps_per_epoch = 367 // BATCH_SIZE,
                                 epochs=epochs,
                                 verbose=2,
                                 validation_data=test_generator,
                                 validation_steps=101 // BATCH_SIZE,
                                 workers=22,
                                 )
    
    return history

In [24]:
model = init_model()
model = train_model(model)

build UNet ...
Epoch 1/100
 - 24s - loss: 1.6108 - iou_score: 0.0828 - val_loss: 1.4643 - val_iou_score: 0.1228
Epoch 2/100
 - 20s - loss: 2.7414 - iou_score: 0.1190 - val_loss: 1.4395 - val_iou_score: 0.0567
Epoch 3/100
 - 20s - loss: 1.3317 - iou_score: 0.1009 - val_loss: 1.5755 - val_iou_score: 0.0259
Epoch 4/100
 - 20s - loss: 1.3847 - iou_score: 0.0969 - val_loss: 1.3107 - val_iou_score: 0.1151
Epoch 5/100
 - 20s - loss: 1.5054 - iou_score: 0.1384 - val_loss: 1.4987 - val_iou_score: 0.1134
Epoch 6/100
 - 20s - loss: 1.3407 - iou_score: 0.1045 - val_loss: 1.2566 - val_iou_score: 0.1262
Epoch 7/100
 - 20s - loss: 1.6094 - iou_score: 0.1033 - val_loss: 1.3161 - val_iou_score: 0.1002
Epoch 8/100
 - 20s - loss: 3.1440 - iou_score: 0.1105 - val_loss: 1.3580 - val_iou_score: 0.1013
Epoch 9/100
 - 20s - loss: 2.3505 - iou_score: 0.0356 - val_loss: 1.3587 - val_iou_score: 0.1056
Epoch 10/100
 - 19s - loss: 1.3664 - iou_score: 0.0954 - val_loss: 1.3620 - val_iou_score: 0.0901
Epoch 11/100
 

KeyboardInterrupt: 

In [22]:
model.fit_generator(train_generator,
                                  steps_per_epoch = 367 // BATCH_SIZE,
                                 epochs=epochs,
                                 verbose=2,
                                 validation_data=test_generator,
                                 validation_steps=101 // BATCH_SIZE,
                                 workers=22,
                                 )

AttributeError: 'History' object has no attribute 'fit_generator'

In [22]:
# model = init_model()
# model.compile('Adam', 
#                  loss='categorical_crossentropy', 
#                  metrics=['categorical_accuracy'])

In [23]:
# history = model.fit_generator(generator=train_generator,
#                                 steps_per_epoch=len(train_ims) // BATCH_SIZE,
#                                 epochs=200,
#                                 verbose=2,
#                                 workers=22)

In [1]:
predict = model.predict_generator(test_generator,
                       steps=3)

NameError: name 'model' is not defined

In [2]:
predict = predict[1]

NameError: name 'predict' is not defined

In [3]:
predict[:, :, 0][0]

NameError: name 'predict' is not defined